<a href="https://colab.research.google.com/github/oldairjsilva/IGTI/blob/main/Projeto_Aplicado_CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Projeto Aplicado

O projeto aplicado tem como desafio coletar os dados financeiros de empresa listada na Bolsa de Valores e comparar com o Valor Intríseco calculado pela fórmula de Graham. 
O objetivo seria verificar o histórico da margem de segurança entre a diferença entre a cotação diária e valor intrínseco calculado pela fórmula de Graham.

#### Base de dados CVM
#### http://dados.cvm.gov.br

#### Configurações iniciais

In [1]:
!pip install wget


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=3ed5ae4dcc1a3f1f267d7b072af85c49780f813d114ceb4d15f156e1af852cfd
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import pandas as pd
import wget
from zipfile import ZipFile
from google.colab.data_table import DataTable
import plotly.graph_objects as go
import numpy as np
from google.colab import drive
import os
from os import path

#### 1. Coletar dados financeiros das empresas na CVM
Acessar dados financeiros de empresas listadas da Bolsa de Valores na CVM e criar arquivos históricos

In [3]:
# Montar drive para armazenar arquivos
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
# Coletar dados financeiros das empresas na CVM
# Função para baixar arquivos CVM
def baixarArquivosCVM(AnoInicio, AnoFim):
  url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
  arquivos_zip = []
  # Criar pasta DADOS no google drive
  if path.exists('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS') == False:
    os.mkdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS')

  os.chdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS')

  # Cria as descrições dos arquivos pelos anos informados
  for ano in range(AnoInicio,AnoFim):
    arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')
  arquivos_zip

  # 1. Baixar arquivos .zip da CVM considerando os anos de início e fim informados
  
  # Faz o download dos arquivos da CVM
  for arq in arquivos_zip:
    wget.download(url_base+arq)

  # 2. Descompactar os arquivos na pasta CVM
  for arq in arquivos_zip:
    ZipFile(arq, 'r').extractall('CVM')

  # 3. Unificar os arquivos por seus tipos. Ex. BPP/DRE
  
  # Cria o diretorio para unificar os dados
  if path.exists('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS') == False:
    os.mkdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS')  

  nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DFC_MD_con', 'DFC_MD_ind', 'DFC_MI_con', 'DFC_MI_ind', 'DMPL_con', 'DMPL_ind', 'DRA_con', 'DRA_ind', 'DRE_con', 'DRE_ind', 'DVA_con', 'DVA_ind']
  for nome in nomes:
    arquivo = pd.DataFrame()

    for ano in range(AnoInicio,AnoFim):
      arquivo = pd.concat([arquivo, pd.read_csv(f'CVM/dfp_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
    arquivo.to_csv(f'UNIFICADOS/dfp_cia_aberta_{nome}_{AnoInicio}-{AnoFim}.csv', index = False)


In [8]:
# Executar a função para baixar arquivos CVM
baixarArquivosCVM(2010, 2023) 

#### 2. 
Acessar dados financeiros para gerar indicadores fundamentalistas e calcular a fórmula de Graham


In [9]:
# Analisar DRE e BPP para calcular indicadores/fórmula de Graham de empresas selecionadas: BCO BRASIL S.A., ENGIE BRASIL ENERGIA S.A. e WEG S.A.
# Função para gerar indicadores fundamentalistas
def gerarIndicadoresFundamentalistas(Empresa):
  # Acesso os arquivos da CVM - DRE e BPP
  dre = pd.read_csv('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS/dfp_cia_aberta_DRE_con_2010-2023.csv')
  bpp = pd.read_csv('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS/dfp_cia_aberta_BPP_con_2010-2023.csv')

  # Seleciona o último exercício informado
  dre = dre[dre['ORDEM_EXERC'] == "ÚLTIMO"]
  bpp = bpp[bpp['ORDEM_EXERC'] == "ÚLTIMO"]

  # Seleciona a empresa a partir do parametro passado
  empresaDRE = dre[dre['DENOM_CIA'].isin([Empresa])]
  empresaBPP = bpp[bpp['DENOM_CIA'].isin([Empresa])]

  # Seleciona as informações de LPA(lucro por ação), Lucro e Patrimonio Liquido a partir das contas contabeis
  contaLPA = empresaDRE[empresaDRE['CD_CONTA'] == '3.99.01.01']
  contaLucro = empresaDRE[empresaDRE['DS_CONTA'].isin(['Lucro/Prejuízo Consolidado do Período', 'Lucro ou Prejuízo Líquido Consolidado do Período'])]
  contaPatrimonioLiquido = empresaBPP[empresaBPP['DS_CONTA'] == 'Patrimônio Líquido Consolidado']

  # Seleciona as informações e renomeia a coluna - LPA
  contaLPA = contaLPA[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaLPA.rename({'VL_CONTA':'LPA'}, axis=1, inplace=True)

  # Seleciona as informações e renomeia a coluna - Lucro
  contaLucro = contaLucro[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaLucro.rename({'VL_CONTA':'LUCRO'}, axis=1, inplace=True)
  contaLucro['LUCRO'] = contaLucro['LUCRO']*1000

  # Seleciona as informações e renomeia a coluna - Patrimonio Liquido
  contaPatrimonioLiquido = contaPatrimonioLiquido[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaPatrimonioLiquido.rename({'VL_CONTA':'PATRIMONIO_LIQUIDO'}, axis=1, inplace=True)
  contaPatrimonioLiquido['PATRIMONIO_LIQUIDO'] = contaPatrimonioLiquido['PATRIMONIO_LIQUIDO']*1000

  # Cria um data frame unificado de contaLPA, contaLucro e contaPatrimonioLiquido
  contaLPA_contaLucro = pd.merge(contaLPA, contaLucro, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')
  contaLPA_contaLucro_contaPatrimonioLiquido = pd.merge(contaLPA_contaLucro, contaPatrimonioLiquido, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')

  # Calcula a quantidade de ações a partir do lucro dividido por lucro por ação
  contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'] = contaLPA_contaLucro_contaPatrimonioLiquido['LUCRO']/contaLPA_contaLucro_contaPatrimonioLiquido['LPA']
  contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'] = contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'].values.astype(np.int64)

  # Calcula o VPA(valor patrimonial por ação) dividindo o valor do patrimonio liquido pela quantidade de ações
  contaLPA_contaLucro_contaPatrimonioLiquido['VPA'] = contaLPA_contaLucro_contaPatrimonioLiquido['PATRIMONIO_LIQUIDO']/contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES']

  # Calcula o valor intrinseco da Formula de Graham
  contaLPA_contaLucro_contaPatrimonioLiquido['VI_GRAHAM'] = np.sqrt(contaLPA_contaLucro_contaPatrimonioLiquido['VPA'] * contaLPA_contaLucro_contaPatrimonioLiquido['LPA'] * 22.5)

  # Cria o data frame indicadoresFundamentalistas
  indicadoresFundamentalistas = contaLPA_contaLucro_contaPatrimonioLiquido

  # Cria o index a partir da DT_REFER
  indicadoresFundamentalistas.index = pd.to_datetime(indicadoresFundamentalistas['DT_REFER'])

  # Remove a coluna DT_REFER depois de criar o indice
  del indicadoresFundamentalistas['DT_REFER']

  # Retorna o data frame com as informações tratadas
  return indicadoresFundamentalistas

In [10]:
gerarIndicadoresFundamentalistas('WEG S.A.')

,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
DT_REFER,,,,,,,
2010-12-31,WEG S.A.,0.84000,5.336340e+08,3.543836e+09,635278571,5.578397,10.267994
2011-12-31,WEG S.A.,0.95000,6.070210e+08,3.906589e+09,638969473,6.113890,11.431728
2012-12-31,WEG S.A.,1.06000,6.648640e+08,4.151626e+09,627230188,6.618983,12.564344
2013-12-31,WEG S.A.,1.36000,8.453040e+08,4.642267e+09,621547058,7.468891,15.117806
2014-12-31,WEG S.A.,1.18365,9.623160e+08,5.139263e+09,813007223,6.321301,12.974963
2015-12-31,WEG S.A.,0.71669,1.165810e+09,6.156060e+09,1626658666,3.784482,7.811962
2016-12-31,WEG S.A.,0.69278,1.127832e+09,6.070832e+09,1627980022,3.729058,7.624099
2017-12-31,WEG S.A.,0.70789,1.140942e+09,6.843307e+09,1611750413,4.245885,8.223530
2018-12-31,WEG S.A.,0.63815,1.344148e+09,7.853257e+09,2106319830,3.728426,7.316702


In [11]:
gerarIndicadoresFundamentalistas('BCO BRASIL S.A.')

,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
DT_REFER,,,,,,,
2010-12-31,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2717109112,20.028248,43.349168
2011-12-31,BCO BRASIL S.A.,4.43,1.273691e+10,6.326922e+10,2875149435,22.005543,46.833775
2012-12-31,BCO BRASIL S.A.,3.93,1.143820e+10,6.989823e+10,2910483460,24.016020,46.082714
2013-12-31,BCO BRASIL S.A.,3.68,1.128883e+10,7.638200e+10,3067617934,24.899449,45.405665
2014-12-31,BCO BRASIL S.A.,4.23,1.334350e+10,8.544004e+10,3154490780,27.085207,50.772380
2015-12-31,BCO BRASIL S.A.,5.03,1.579804e+10,8.622999e+10,3140763220,27.455108,55.742550
2016-12-31,BCO BRASIL S.A.,2.52,8.659577e+09,9.007669e+10,3436340079,26.212974,38.552245
2017-12-31,BCO BRASIL S.A.,3.82,1.227530e+10,1.012384e+11,3213430104,31.504786,52.036875
2018-12-31,BCO BRASIL S.A.,4.96,1.508610e+10,1.045400e+11,3041552620,34.370594,61.933499


In [12]:
gerarIndicadoresFundamentalistas('ENGIE BRASIL ENERGIA S.A.')

,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
DT_REFER,,,,,,,
2010-12-31,ENGIE BRASIL ENERGIA S.A.,1.85624,1.211649e+09,5.072684e+09,652743718,7.771326,18.015896
2011-12-31,ENGIE BRASIL ENERGIA S.A.,2.21850,1.448106e+09,5.450258e+09,652741041,8.349801,20.415454
2012-12-31,ENGIE BRASIL ENERGIA S.A.,2.29788,1.499923e+09,5.462840e+09,652742092,8.369063,20.801438
2013-12-31,ENGIE BRASIL ENERGIA S.A.,2.20109,1.436747e+09,5.364611e+09,652743413,8.218560,20.174744
2014-12-31,ENGIE BRASIL ENERGIA S.A.,2.11891,1.383105e+09,5.654949e+09,652743627,8.663354,20.323127
2015-12-31,ENGIE BRASIL ENERGIA S.A.,2.30000,1.501303e+09,6.642136e+09,652740434,10.175769,22.947681
2016-12-31,ENGIE BRASIL ENERGIA S.A.,2.37200,1.548301e+09,6.614394e+09,652740725,10.133264,23.255371
2017-12-31,ENGIE BRASIL ENERGIA S.A.,3.07098,2.004559e+09,6.834725e+09,652742447,10.470784,26.897960
2018-12-31,ENGIE BRASIL ENERGIA S.A.,2.83648,2.315407e+09,6.320577e+09,816295901,7.742997,22.229806


In [ ]:
#empresas = dre[['DENOM_CIA', 'CD_CVM']].drop_duplicates().set_index('CD_CVM')
#DataTable(empresas)

In [ ]:
#DataTable(empresaDRE[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))
#DataTable(empresaBPP[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))

#### 3. Baixar cotações diárias através do Yahoo Finance

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import yfinance as yf

In [ ]:
pricesBBAS3 = yf.download('BBAS3.SA', start='2011-01-01')[['Close', 'Adj Close']]
pricesWEGE3 = yf.download('WEGE3.SA', start='2011-01-01')[['Close', 'Adj Close']]
pricesEGIE3 = yf.download('EGIE3.SA', start='2011-01-01')[['Close', 'Adj Close']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
pricesEGIE3.head()

,Close,Adj Close
Date,,
2011-01-03,21.440001,10.339413
2011-01-04,21.160000,10.204383
2011-01-05,21.520000,10.377994
2011-01-06,21.799999,10.513021
2011-01-07,21.879999,10.551603


#### 4. Juntar as cotações diárias com os indicadores fundamentalistas

In [ ]:
indicadoresBBAS3 = pricesBBAS3.join(gerarIndicadoresFundamentalistas('BCO BRASIL S.A.'), how = 'outer')
indicadoresWEGE3 = pricesWEGE3.join(gerarIndicadoresFundamentalistas("WEG S.A."), how = 'outer')
indicadoresEGIE3 = pricesEGIE3.join(gerarIndicadoresFundamentalistas("ENGIE BRASIL ENERGIA S.A."), how = 'outer')

In [ ]:
indicadoresEGIE3.head()

In [ ]:
# Preencher os dados fundamentalistas a partir da data de disponibilização para o ano seguinte
indicadoresBBAS3.fillna(method='ffill', inplace = True)
indicadoresWEGE3.fillna(method='ffill', inplace = True)
indicadoresEGIE3.fillna(method='ffill', inplace = True)

In [ ]:
indicadoresEGIE3

In [ ]:
# Remover as linhas nulas referente aos dados de Fechamento(Close) e Fechamento ajustado(Adj Close)
indicadoresBBAS3.dropna(inplace = True)
indicadoresWEGE3.dropna(inplace = True)
indicadoresEGIE3.dropna(inplace = True)

In [ ]:
indicadoresEGIE3

In [ ]:
indicadores = pd.concat([indicadoresBBAS3, indicadoresWEGE3, indicadoresEGIE3])

In [ ]:
indicadores

In [ ]:
# Exportar os dados em formato csv para fazer dashboard pelo Google Data Studio
indicadores.to_csv('indicadores_BBAS3_EGIE3_WEGE3.csv', index = True, sep = ';', decimal = '.')

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['Close'],  name='BBAS3'))
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['Adj Close'],  name='BBAS3_Ajustado'))

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['Close'],  name='EGIE3'))
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['Adj Close'],  name='EGIE3_Ajustado'))

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['Close'],  name='WEGE3'))
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['Adj Close'],  name='WEGE3_Ajustado'))